In [ ]:
from crosscheck.crosscheck import crosscheck

In [ ]:
groups = [
    [
        'https://doi.org/10.3389/fphys.2012.00450'
    ],
    [
        'https://doi.org/10.1016/j.jneumeth.2007.03.024'
    ]
]

In [ ]:
results, seed = crosscheck(groups)

In [ ]:
print(f'Found {len(results)} papers:')

for paper_id in results:
    print(f'https://semanticscholar.org/paper/{paper_id}')

In [ ]:
for i, group in enumerate(seed):
    print(f'Seed papers - Group {i+1}:')
    for paper_id in group:
        print(f'https://semanticscholar.org/paper/{paper_id}')